# **Using four recurrent neural network architecture with support vector machine to detect and differentiate between false and real news article information in a text-based data set.**

**RNN(LSTM, GRU, Bidirectional LSTM and Bidirectional GRU)-SVM models**

This product has 9 stages to detect and analysis true and false information in news articles. These nine stages are Load the Data Set, Text Preprocessing, Splitting the Data Set, Tokenisation, Label Encoding, Pretrained GloVe word embedding, Training the hybrid models (LSTM-SVM, GRU-SVM, Bi_LSTM-SVM, Bi_GRU-SVM), Accuracy Analysis and Testing Class.

**Importing the necessary libraries**

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
import string
import itertools

from wordcloud import WordCloud, ImageColorGenerator

import nltk 
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import RNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation
from tensorflow.keras.losses import Hinge
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

%matplotlib inline

**Activating the Kaggle OS path for data set retrieval**

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Using the TensorFlow-GPU version instead of the CPU version**

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

In [ ]:
tf.config.experimental.set_memory_growth(gpus[0], enable=True)
print("Tensorflow Version",tf.__version__)

# Stage 1: Load the Data Set

Load the true news data set

In [ ]:
true = pd.read_csv('../input/fake-and-real-news-dataset/True.csv', encoding='latin', header=None)
true.head(5)

Load the fake news data set

In [ ]:
false = pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv', encoding='latin', header=None)
false.head(5)

**Data set Examination**

Dropping the first row of useless text because they are the title of the columns

In [ ]:
true = true.drop([0], axis=0)
true.head(5)

In [ ]:
false = false.drop([0], axis=0)
false.head(5)

Adding the correct title for each column

In [ ]:
true.columns = ['title', 'text', 'subject','date']
true.head(5)

In [ ]:
false.columns = ['title', 'text', 'subject','date']
false.head(5)

Check both data sets to see if it is identical

In [ ]:
print(false.shape)
print(true.shape)

**Displaying the most used words in the positive and negative sentiment labels**

Positive words

In [ ]:
text = " ".join(str(each) for each in true.text.unique())
wordcloud = WordCloud(max_words=1000, background_color="black").generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
#plt.savefig('img2/data_set_2_full_positive_words.png', format='png')
plt.show()

Negative words

In [ ]:
text = " ".join(str(each) for each in false.text.unique())
wordcloud = WordCloud(max_words=1000, background_color="black").generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
#plt.savefig('img2/data_set_2_full_negative_words.png', format='png')
plt.show()

Applying integer values to the true and false news data sets

In [ ]:
true['sentiment'] = 1
false['sentiment'] = 0

In [ ]:
true.head(5)

In [ ]:
false.head(5)

**Reducing the data set**

* Reasons: Hardware GPU power limitation and creating an equal data set to compare it against another product prototype.

Dividing the data set into an equal amount

In [ ]:
true_data = true.iloc[:int(len(true)/2.1417)]
false_data = false.iloc[:int(len(false)/2.3481)]
print(len(true_data), len(false_data))

Joining both the data sets back together

In [ ]:
news = pd.concat([true_data,false_data]) 
len(news)

Viewing the connected data set

In [ ]:
news

Data set details

In [ ]:
list(news.columns.values)

In [ ]:
print(news.shape)

In [ ]:
news.shape

In [ ]:
news.count()

In [ ]:
news.dtypes

In [ ]:
sentiment_counts = news.sentiment.value_counts()
print(sentiment_counts)

In [ ]:
text_counts = news.text.value_counts()
print(text_counts)

In [ ]:
news.describe()

Visual representation of the data

In [ ]:
news['sentiment'].value_counts().plot(kind='bar')
#plt.savefig('img2/data_set_2_news_bar_chart.png', format='png')

In [ ]:
news['sentiment'].value_counts().plot(kind='pie')
#plt.savefig('img2/data_set_2_news_pie_chart.png', format='png')

Text and sentiment columns are kept while the rest are dropped

In [ ]:
data=news.drop(["date","title","subject"],axis=1)

In [ ]:
data

Analysing the data set shape

In [ ]:
encoded_data = data

In [ ]:
print(data.shape)

Assigning label numbers with positive and negative values

In [ ]:
labels_to_sentiment = {0:"negative", 1:"positive"}
def label_decoder(label):
    return labels_to_sentiment[label]
    encoded_data.sentiment = encoded_data.sentiment.apply(lambda x: label_decoder(x))
encoded_data.head(5)

Checking the distribution of the data after the reduction phase

In [ ]:
val_count = encoded_data.sentiment.value_counts()

plt.figure(figsize=(8,4))
plt.bar(val_count.index, val_count.values)
plt.title("1.6 Million Twitter Tweets Sentiment Data Distribution")
#plt.savefig('img2/data_set_2_news_distribution_chart.png', format='png')

Picking the right naming convention for the data set before it goes through the text preprocessing stage

In [ ]:
reduced_data = encoded_data
reduced_data.head(5)

In [ ]:
cleaned_data = reduced_data
cleaned_data.head(5)

# Stage 2: Text Preprocessing

Removing URL and HTML

In [ ]:
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

Removing any leftovers of unnecessary texts

In [ ]:
text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"    

def text_cleaning(text):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    return text

Removing integers

In [ ]:
def number_cleaning(text):
    text = ''.join(c for c in text if not c.isdigit())
    return text

Removing emojis

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile(
    "["
        u"\U0001F600-\U0001F64F" #emoticons
        u"\U0001F300-\U0001F5FF" #symbols & pictographs
        u"\U0001F680-\U0001F6FF" #transport & map symbols
        u"\U0001F1E0-\U0001F1FF" #FLAGS on (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

Removing punctuation

In [ ]:
def remove_punctuation(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

Stemming the text using the Porter version

In [ ]:
def stemming_words(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

Removing stopwords

In [ ]:
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    
    return " ".join(text)

**Applying text preprocessing**

In [ ]:
cleaned_data["text"] = cleaned_data.text.apply(lambda x: remove_URL(x))
cleaned_data["text"] = cleaned_data.text.apply(lambda x: remove_html(x))
cleaned_data["text"] = cleaned_data.text.apply(lambda x: text_cleaning(x))
cleaned_data["text"] = cleaned_data.text.apply(lambda x: number_cleaning(x))
cleaned_data["text"] = cleaned_data.text.apply(lambda x: remove_emoji(x))
cleaned_data["text"] = cleaned_data.text.apply(lambda x: remove_punctuation(x))
cleaned_data["text"] = cleaned_data.text.apply(lambda x: stemming_words(x))
cleaned_data["text"] = cleaned_data["text"].apply(remove_stopwords)

Viewing the cleaned data

In [ ]:
cleaned_data.head(5)

In [ ]:
cleaned_data.text

# Stage 3: Splitting the Data Set

In [ ]:
train_size = 0.8

Splitting the data set into 80% training and 20% testing set then randomise it for balance purpose

In [ ]:
train_data, test_data = train_test_split(cleaned_data, test_size=1-train_size,
                                         random_state=7)

print("Train Data size:", len(train_data))
print("Test Data size", len(test_data))

In [ ]:
train_data.head(10)

# Stage 4: Tokenisation

**Hyper parameters**

Description:

* 100,000 will be the vocabulary range.
* 500 words was chosen as the max length due to GPU power limitation because the length of each news article is around 300-19,946 words long.

In [ ]:
vocabulary_size = 100000
max_word_length = 500
oov_tk = "<OOV>"
trunc_type='post'
padding_type='post'

**Tokenising the training data set**

Description:

* OOV means out of vocabulary. The tokenizer will create a token for each word and replace each word it does not recognise with the 'OOV' token instead.
* num_words stores the maximum number of vocabulary words.
* word_index is a dictionary containing a key for a word and a value which is a token linked specifically to a word.

In [ ]:
tokenizer = Tokenizer(num_words = vocabulary_size, oov_token = oov_tk)
tokenizer.fit_on_texts(train_data.text)

word_index = tokenizer.word_index  
vocab_size = len(tokenizer.word_index) + 1

print("Vocabulary Size :", vocab_size)

**Sequences**

Description:

* Placing the train and test data sets into sequences. This creates sequences of tokens representing each sentence.

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_data.text) 
test_sequences = tokenizer.texts_to_sequences(test_data.text)

**Padding**

Description:

* Padding is used to handle sentences with different lengths.

In [ ]:
x_train = pad_sequences(train_sequences, maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
x_test = pad_sequences(test_sequences, maxlen=max_word_length, padding=padding_type, truncating=trunc_type)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

# Stage 5: Label Encoding

In [ ]:
labels = train_data.sentiment.unique().tolist()

**Encoding the sentiment labels**

In [ ]:
encoder = LabelEncoder()
encoder.fit(train_data.sentiment.to_list())

y_train = encoder.transform(train_data.sentiment.to_list())
y_test = encoder.transform(test_data.sentiment.to_list())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

**Viewing important parameters data**

In [ ]:
vocabulary_size =len(word_index)
print('The size of the news dataset vocabulary is: ', vocabulary_size)

In [ ]:
training_sequence = x_train.shape[1]
print('The length of training sequence is: ', training_sequence)

In [ ]:
print('The length of testing sequence is: ', x_test.shape[1])

# Stage 6: Pretrained GloVe word embedding

**Loading the GloVe word embedding 6-billion and 300-dimensional text**

In [ ]:
embeddings_index = {}

with open('../input/glove6b/glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print('Found %s word vectors.' % len(embeddings_index))

**Unique Values**

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
embeddings_index

**Creating the embedding matrix**

In [ ]:
num_words = len(word_index) + 1
embedding_dimension = 300

embedding_matrix = np.zeros((vocab_size, embedding_dimension))

for word, i in word_index.items():
    if i < num_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix

**Testing out the embedding matrix**

In [ ]:
word_index["good"]

In [ ]:
embeddings_index.get("good")

In [ ]:
(embedding_matrix[174] == embeddings_index.get("good")).all()

**Analysing the training and testing data sets shapes**

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
print(x_test.shape)
print(y_test.shape)

**Checking to see where the training of the model will be done (GPU or CPU)**

In [ ]:
print("Training on GPU...") if tf.test.is_gpu_available() else print("Training on CPU...")

# Stage 7: Training the four RNN-SVM Models

Important Note: The Losses Hinge function converts the encoded labels from 0 to 1 which are negative and positive to -1 and 1.

# Training the LSTM-SVM Model

**Creating the LSTM-SVM Model**

In [ ]:
LSTM_SVM_model = tf.keras.Sequential([

    tf.keras.layers.Embedding(vocab_size, embedding_dimension, 
                            weights=[embedding_matrix], 
                            input_length=max_word_length, 
                            trainable=False),
    
    tf.keras.layers.SpatialDropout1D(0.40),
    tf.keras.layers.LSTM((128), return_sequences=True),
    tf.keras.layers.Dropout(0.20),
    tf.keras.layers.LSTM((128)),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, kernel_regularizer=l2(0.01)),
    tf.keras.layers.Activation('linear')
])

In [ ]:
LSTM_SVM_model.compile(loss=tf.keras.losses.Hinge(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
print(LSTM_SVM_model.summary())

**Training the LSTM-SVM model**

In [ ]:
number_of_epochs = 10
batch_size = 256

LSTM_SVM_history = LSTM_SVM_model.fit(x_train, y_train, 
                                    epochs = number_of_epochs,
                                    batch_size = batch_size,
                   validation_data=(x_test, y_test), 
                                    verbose=1)

**Plotting the LSTM-SVM train graph**

In [ ]:
plt.plot(LSTM_SVM_history.history['accuracy'])
plt.plot(LSTM_SVM_history.history['val_accuracy'])
plt.title('LSTM-SVM Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.savefig('img2/data_set_2_LSTM_SVM_history_model_accuracy.png', format='png')
plt.show()


plt.plot(LSTM_SVM_history.history['loss'])
plt.plot(LSTM_SVM_history.history['val_loss'])
plt.title('LSTM-SVM Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.savefig('img2/data_set_2_LSTM_SVM_model_loss.png', format='png')
plt.show()

**Saving the LSTM-SVM model**

In [ ]:
#LSTM_SVM_model.save('saved_model2/LSTM_SVM_Model')

**Loading the LSTM-SVM model**

In [ ]:
#LSTM_SVM_saved_model = tf.keras.models.load_model('saved_model2/LSTM_SVM_Model')

# Training the Bidirectional LSTM-SVM Model

**Creating the Bidirectional LSTM-SVM Model**

In [ ]:
BILSTM_SVM_model = tf.keras.Sequential([

    tf.keras.layers.Embedding(vocab_size, embedding_dimension, 
                            weights=[embedding_matrix], 
                            input_length=max_word_length, 
                            trainable=False),
    
    tf.keras.layers.SpatialDropout1D(0.40),
    tf.keras.layers.Bidirectional(tf.keras.layers.RNN(tf.keras.layers.LSTMCell(128), return_sequences=True)),
    tf.keras.layers.Dropout(0.20),
    tf.keras.layers.Bidirectional(tf.keras.layers.RNN(tf.keras.layers.LSTMCell(128))),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, kernel_regularizer=l2(0.01)),
    tf.keras.layers.Activation('linear')
])

In [ ]:
BILSTM_SVM_model.compile(loss=tf.keras.losses.Hinge(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
print(BILSTM_SVM_model.summary())

**Training the Bidirectional LSTM-SVM model**

In [ ]:
number_of_epochs = 10
batch_size = 256

BILSTM_SVM_history = BILSTM_SVM_model.fit(x_train, y_train, 
                                    epochs = number_of_epochs,
                                    batch_size = batch_size,
                   validation_data=(x_test, y_test), 
                                    verbose=1)

**Plotting the Bidirectional LSTM-SVM train graph**

In [ ]:
plt.plot(BILSTM_SVM_history.history['accuracy'])
plt.plot(BILSTM_SVM_history.history['val_accuracy'])
plt.title('BILSTM-SVM Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.savefig('img2/data_set_2_BILSTM_SVM_model_accuracy.png', format='png')
plt.show()


plt.plot(BILSTM_SVM_history.history['loss'])
plt.plot(BILSTM_SVM_history.history['val_loss'])
plt.title('BILSTM-SVM Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.savefig('img2/data_set_2_BILSTM_SVM_model_loss.png', format='png')
plt.show()

**Saving the Bidirectional LSTM-SVM model**

In [ ]:
#BILSTM_SVM_model.save('saved_model2/BILSTM_SVM_Model')

**Loading the Bidirectional LSTM-SVM model**

In [ ]:
#BILSTM_SVM_saved_model = tf.keras.models.load_model('saved_model2/BILSTM_SVM_Model')

# Training the GRU-SVM Model

**Creating the GRU-SVM Model**

In [ ]:
GRU_SVM_model = tf.keras.Sequential([

    tf.keras.layers.Embedding(vocab_size, embedding_dimension, 
                            weights=[embedding_matrix], 
                            input_length=max_word_length, 
                            trainable=False),
    
    tf.keras.layers.SpatialDropout1D(0.40),
    tf.keras.layers.GRU((128), return_sequences=True),
    tf.keras.layers.Dropout(0.20),
    tf.keras.layers.GRU((128)),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, kernel_regularizer=l2(0.01)),
    tf.keras.layers.Activation('linear')
])

In [ ]:
GRU_SVM_model.compile(loss=tf.keras.losses.Hinge(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
print(GRU_SVM_model.summary())

**Training the GRU-SVM model**

In [ ]:
number_of_epochs = 10
batch_size = 256

GRU_SVM_history = GRU_SVM_model.fit(x_train, y_train, 
                                    epochs = number_of_epochs,
                                    batch_size = batch_size,
                   validation_data=(x_test, y_test), 
                                    verbose=1)

**Plotting the GRU-SVM train graph**

In [ ]:
plt.plot(GRU_SVM_history.history['accuracy'])
plt.plot(GRU_SVM_history.history['val_accuracy'])
plt.title('GRU-SVM Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.savefig('img2/data_set_2_GRU_SVM_model_accuracy.png', format='png')
plt.show()

plt.plot(GRU_SVM_history.history['loss'])
plt.plot(GRU_SVM_history.history['val_loss'])
plt.title('GRU-SVM Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.savefig('img2/data_set_2_GRU_SVM_model_loss.png', format='png')
plt.show()

**Saving the GRU-SVM model**

In [ ]:
#GRU_SVM_model.save('saved_model2/GRU_SVM_Model')

**Loading the GRU-SVM model**

In [ ]:
#GRU_SVM_saved_model = tf.keras.models.load_model('saved_model2/GRU_SVM_Model')

# Training the Bidirectional GRU-SVM Model

**Creating the Bidirectional GRU-SVM Model**

In [ ]:
BIGRU_SVM_model = tf.keras.Sequential([

    tf.keras.layers.Embedding(vocab_size, embedding_dimension, 
                            weights=[embedding_matrix], 
                            input_length=max_word_length, 
                            trainable=False),
    
    tf.keras.layers.SpatialDropout1D(0.40),
    tf.keras.layers.Bidirectional(tf.keras.layers.RNN(tf.keras.layers.GRUCell(128), return_sequences=True)),
    tf.keras.layers.Dropout(0.20),
    tf.keras.layers.Bidirectional(tf.keras.layers.RNN(tf.keras.layers.GRUCell(128))),
    tf.keras.layers.Dropout(0.30),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, kernel_regularizer=l2(0.01)),
    tf.keras.layers.Activation('linear')
])

In [ ]:
BIGRU_SVM_model.compile(loss=tf.keras.losses.Hinge(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
print(BIGRU_SVM_model.summary())

**Training the Bidirectional GRU-SVM model**

In [ ]:
number_of_epochs = 10
batch_size = 256

BIGRU_SVM_history = BIGRU_SVM_model.fit(x_train, y_train, 
                                    epochs = number_of_epochs,
                                    batch_size = batch_size,
                   validation_data=(x_test, y_test), 
                                    verbose=1)

**Plotting the Bidirectional GRU-SVM train graph**

In [ ]:
plt.plot(BIGRU_SVM_history.history['accuracy'])
plt.plot(BIGRU_SVM_history.history['val_accuracy'])
plt.title('BIGRU-SVM Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.savefig('img2/data_set_2_BIGRU_SVM_model_accuracy.png', format='png')
plt.show()

plt.plot(BIGRU_SVM_history.history['loss'])
plt.plot(BIGRU_SVM_history.history['val_loss'])
plt.title('BIGRU-SVM Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
#plt.savefig('img2/data_set_2_BIGRU_SVM_model_loss.png', format='png')
plt.show()

**Saving the Bidirectional GRU-SVM model**

In [ ]:
#BIGRU_SVM_model.save('saved_model2/BIGRU_SVM_Model')

**Loading the Bidirectional GRU-SVM model**

In [ ]:
#BIGRU_SVM_saved_model = tf.keras.models.load_model('saved_model2/BIGRU_SVM_Model')

# Stage 8: Accuracy Analysis

# LSTM-SVM Accuracy Analysis

**Test accuracy and loss**

In [ ]:
test_loss, test_acc = LSTM_SVM_model.evaluate(x_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

**Testing the hybrid model against the testing data set**

In [ ]:
LSTM_SVM_scores = LSTM_SVM_model.predict(x_test, verbose=1, batch_size=256)

In [ ]:
LSTM_SVM_scores

In [ ]:
LSTM_SVM_y_pred=np.where(LSTM_SVM_scores>0.5,1,0)

In [ ]:
LSTM_SVM_y_pred

**Accuracy**

In [ ]:
print(accuracy_score(y_test, LSTM_SVM_y_pred))

**Classification report (Precision, Recall, F1-Score and Accuracy)**

In [ ]:
print(classification_report(y_test, LSTM_SVM_y_pred))

**Confusion Matrix**

In [ ]:
LSTM_SVM_CM=confusion_matrix(y_test, LSTM_SVM_y_pred)
print(LSTM_SVM_CM)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalised confusion matrix")
    else:
        print('Confusion matrix, without normalisation')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
categories = ['Negative','Positive']
LSTM_SVM_cnf_matrix = confusion_matrix(y_test, LSTM_SVM_y_pred)
plot_confusion_matrix(cm=LSTM_SVM_cnf_matrix, classes=categories, title='LSTM-SVM Confusion Matrix')
#plt.savefig('img2/data_set_2_LSTM_SVM_CM.png', format='png')
plt.show()

# Bidirectional LSTM-SVM Accuracy Analysis

**Test accuracy and loss**

In [ ]:
test_loss, test_acc = BILSTM_SVM_model.evaluate(x_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

**Testing the hybrid model against the testing data set**

In [ ]:
BILSTM_SVM_scores = BILSTM_SVM_model.predict(x_test, verbose=1, batch_size=256)

In [ ]:
BILSTM_SVM_scores

In [ ]:
BILSTM_SVM_y_pred=np.where(BILSTM_SVM_scores>0.5,1,0)

In [ ]:
BILSTM_SVM_y_pred

**Accuracy**

In [ ]:
print(accuracy_score(y_test, BILSTM_SVM_y_pred))

**Classification report (Precision, Recall, F1-Score and Accuracy)**

In [ ]:
print(classification_report(y_test, BILSTM_SVM_y_pred))

**Confusion Matrix**

In [ ]:
BILSTM_SVM_CM=confusion_matrix(y_test, BILSTM_SVM_y_pred)
print(BILSTM_SVM_CM)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalised confusion matrix")
    else:
        print('Confusion matrix, without normalisation')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
categories = ['Negative','Positive']
BILSTM_SVM_cnf_matrix = confusion_matrix(y_test, BILSTM_SVM_y_pred)
plot_confusion_matrix(cm=BILSTM_SVM_cnf_matrix, classes=categories, title='BILSTM-SVM Confusion Matrix')
#plt.savefig('img2/data_set_2_BILSTM_SVM_CM.png', format='png')
plt.show()

# GRU-SVM Accuracy Analysis

**Test accuracy and loss**

In [ ]:
test_loss, test_acc = GRU_SVM_model.evaluate(x_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

**Testing the hybrid model against the testing data set**

In [ ]:
GRU_SVM_scores = GRU_SVM_model.predict(x_test, verbose=1, batch_size=256)

In [ ]:
GRU_SVM_scores

In [ ]:
GRU_SVM_y_pred=np.where(GRU_SVM_scores>0.5,1,0)

In [ ]:
GRU_SVM_y_pred

**Accuracy**

In [ ]:
print(accuracy_score(y_test, GRU_SVM_y_pred))

**Classification report (Precision, Recall, F1-Score and Accuracy)**

In [ ]:
print(classification_report(y_test, GRU_SVM_y_pred))

**Confusion Matrix**

In [ ]:
GRU_SVM_CM=confusion_matrix(y_test, GRU_SVM_y_pred)
print(GRU_SVM_CM)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalised confusion matrix")
    else:
        print('Confusion matrix, without normalisation')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
categories = ['Negative','Positive']
GRU_SVM_cnf_matrix = confusion_matrix(y_test, GRU_SVM_y_pred)
plot_confusion_matrix(cm=GRU_SVM_cnf_matrix, classes=categories, title='GRU-SVM Confusion Matrix')
#plt.savefig('img2/data_set_2_GRU_SVM_CM.png', format='png')
plt.show()

# Bidirectional GRU-SVM Accuracy Analysis

**Test accuracy and loss**

In [ ]:
test_loss, test_acc = BIGRU_SVM_model.evaluate(x_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

**Testing the hybrid model against the testing data set**

In [ ]:
BIGRU_SVM_scores = BIGRU_SVM_model.predict(x_test, verbose=1, batch_size=256)

In [ ]:
BIGRU_SVM_scores

In [ ]:
BIGRU_SVM_y_pred=np.where(BIGRU_SVM_scores>0.5,1,0)

In [ ]:
BIGRU_SVM_y_pred

**Accuracy**

In [ ]:
print(accuracy_score(y_test, BIGRU_SVM_y_pred))

**Classification report (Precision, Recall, F1-Score and Accuracy)**

In [ ]:
print(classification_report(y_test, BIGRU_SVM_y_pred))

**Confusion Matrix**

In [ ]:
BIGRU_SVM_CM=confusion_matrix(y_test, BIGRU_SVM_y_pred)
print(BIGRU_SVM_CM)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalised confusion matrix")
    else:
        print('Confusion matrix, without normalisation')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
categories = ['Negative','Positive']
BIGRU_SVM_cnf_matrix = confusion_matrix(y_test, BIGRU_SVM_y_pred)
plot_confusion_matrix(cm=BIGRU_SVM_cnf_matrix, classes=categories, title='BIGRU-SVM Confusion Matrix')
#plt.savefig('img2/data_set_2_BIGRU_SVM_CM.png', format='png')
plt.show()

# Stage 9: Testing Class

Four user created English sentences will be used to evaluate if the hybrid models is able to distinguish between true and false news article.

* 1st sentence is a true news article from the true data set
* 2nd sentence is a false news article from the fake data set
* 3rd sentence is a true news article from the REUTERS news website (https://www.reuters.com/world/china/uks-johnson-tells-china-we-believe-rule-sea-2021-05-21/)
* 4th sentence is a false news article from the FactCheck.org website (https://www.factcheck.org/2020/02/fake-coronavirus-cures-part-2-garlic-isnt-a-cure/)

# LSTM-SVM Testing Class

**Testing the LSTM-SVM model by using a basic test function**

In [ ]:
test_sentence1 = [ "WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a â€œfiscal conservativeâ€ on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBSâ€™ â€œFace the Nation,â€ drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense â€œdiscretionaryâ€ spending on programs that support education, scientific research, infrastructure, public health and environmental protection. â€œThe (Trump) administration has already been willing to say: â€˜Weâ€™re going to increase non-defense discretionary spending ... by about 7 percent,â€™â€ Meadows, chairman of the small but influential House Freedom Caucus, said on the program. â€œNow, Democrats are saying thatâ€™s not enough, we need to give the government a pay raise of 10 to 11 percent. For a fiscal conservative, I donâ€™t see where the rationale is. ... Eventually you run out of other peopleâ€™s money,â€ he said. Meadows was among Republicans who voted in late December for their partyâ€™s debt-financed tax overhaul, which is expected to balloon the federal budget deficit and add about $1.5 trillion over 10 years to the $20 trillion national debt. â€œItâ€™s interesting to hear Mark talk about fiscal responsibility,â€ Democratic U.S. Representative Joseph Crowley said on CBS. Crowley said the Republican tax bill would require the  United States to borrow $1.5 trillion, to be paid off by future generations, to finance tax cuts for corporations and the rich. â€œThis is one of the least ... fiscally responsible bills weâ€™ve ever seen passed in the history of the House of Representatives. I think weâ€™re going to be paying for this for many, many years to come,â€ Crowley said. Republicans insist the tax package, the biggest U.S. tax overhaul in more than 30 years,  will boost the economy and job growth. House Speaker Paul Ryan, who also supported the tax bill, recently went further than Meadows, making clear in a radio interview that welfare or â€œentitlement reform,â€ as the party often calls it, would be a top Republican priority in 2018. In Republican parlance, â€œentitlementâ€ programs mean food stamps, housing assistance, Medicare and Medicaid health insurance for the elderly, poor and disabled, as well as other programs created by Washington to assist the needy. Democrats seized on Ryanâ€™s early December remarks, saying they showed Republicans would try to pay for their tax overhaul by seeking spending cuts for social programs. But the goals of House Republicans may have to take a back seat to the Senate, where the votes of some Democrats will be needed to approve a budget and prevent a government shutdown. Democrats will use their leverage in the Senate, which Republicans narrowly control, to defend both discretionary non-defense programs and social spending, while tackling the issue of the â€œDreamers,â€ people brought illegally to the country as children. Trump in September put a March 2018 expiration date on the Deferred Action for Childhood Arrivals, or DACA, program, which protects the young immigrants from deportation and provides them with work permits. The president has said in recent Twitter messages he wants funding for his proposed Mexican border wall and other immigration law changes in exchange for agreeing to help the Dreamers. Representative Debbie Dingell told CBS she did not favor linking that issue to other policy objectives, such as wall funding. â€œWe need to do DACA clean,â€ she said.  On Wednesday, Trump aides will meet with congressional leaders to discuss those issues. That will be followed by a weekend of strategy sessions for Trump and Republican leaders on Jan. 6 and 7, the White House said. Trump was also scheduled to meet on Sunday with Florida Republican Governor Rick Scott, who wants more emergency aid. The House has passed an $81 billion aid package after hurricanes in Florida, Texas and Puerto Rico, and wildfires in California. The package far exceeded the $44 billion requested by the Trump administration. The Senate has not yet voted on the aid.", 
                     "Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t even allow him to rise above the gutter long enough to wish the American citizens a happy new year!  Bishop Talbert Swan (@TalbertSwan) December 31, 2017no one likes you  Calvin (@calvinstowell) December 31, 2017Your impeachment would make 2018 a great year for America, but I ll also accept regaining control of Congress.  Miranda Yaver (@mirandayaver) December 31, 2017Do you hear yourself talk? When you have to include that many people that hate you you have to wonder? Why do the they all hate me?  Alan Sandoval (@AlanSandoval13) December 31, 2017Who uses the word Haters in a New Years wish??  Marlene (@marlene399) December 31, 2017You can t just say happy new year?  Koren pollitt (@Korencarpenter) December 31, 2017Here s Trump s New Year s Eve tweet from 2016.Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don t know what to do. Love!  Donald J. Trump (@realDonaldTrump) December 31, 2016This is nothing new for Trump. He s been doing this for years.Trump has directed messages to his  enemies  and  haters  for New Year s, Easter, Thanksgiving, and the anniversary of 9/11. pic.twitter.com/4FPAe2KypA  Daniel Dale (@ddale8) December 31, 2017Trump s holiday tweets are clearly not presidential.How long did he work at Hallmark before becoming President?  Steven Goodine (@SGoodine) December 31, 2017He s always been like this . . . the only difference is that in the last few years, his filter has been breaking down.  Roy Schulze (@thbthttt) December 31, 2017Who, apart from a teenager uses the term haters?  Wendy (@WendyWhistles) December 31, 2017he s a fucking 5 year old  Who Knows (@rainyday80) December 31, 2017So, to all the people who voted for this a hole thinking he would change once he got into power, you were wrong! 70-year-old men don t change and now he s a year older.Photo by Andrew Burton/Getty Images.",
                     "UNITED KINGDOM (Reuters) - The first operational deployment of Britain's flagship aircraft carrier HMS Queen Elizabeth to Asia will show countries such as China that Britain believes in the international law of the sea, Prime Minister Boris Johnson said on Friday. Johnson said the Carrier Strike Group, which will interact with more than 40 nations on the deployment through the Mediterranean, Indian Ocean and South China Sea to Japan, would project Britain's values as well as its military capabilities. One of the things we'll be doing clearly is showing to our friends in China that we believe in the international law of the sea, and in a confident but not a confrontational way, we will be vindicating that point, he told broadcasters on board the carrier in Portsmouth Naval Base, south England. We don't want to antagonise anybody, but we do think that the United Kingdom plays a very important role, with friends and partners, the Americans, the Dutch, the Australians, the Indians many, many others, in upholding the rule of law, the international rules-based system on which we all depend.",
                     "Good news, Wuhan’s corona virus can be cured by one bowl of freshly boiled garlic water. Old Chinese doctor has proven it’s efficacy.  Many patients has also proven this to be effective. Eight (8) cloves of chopped garlics add seven (7)cups of water and bring to boil.,  Eat and drink the boiled garlic water, overnight improvement and healing. Glad to share this."]


#Converting the sentences to sequences using a tokenizer
test_sequences1 = tokenizer.texts_to_sequences(test_sentence1)

#Padding the new sequences to make them have the same dimensions
test_padded1 = pad_sequences(test_sequences1, maxlen = max_word_length, 
                             padding = padding_type, truncating = trunc_type)

#Testing out the new padded data on a trained LSTM-SVM model
test_padded1 = np.array(test_padded1)
print(LSTM_SVM_model.predict(test_padded1))

# Bidirectional LSTM-SVM Testing Class

**Testing the Bidirectional LSTM-SVM model by using a basic test function**

In [ ]:
test_sentence2 = [ "WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a â€œfiscal conservativeâ€ on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBSâ€™ â€œFace the Nation,â€ drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense â€œdiscretionaryâ€ spending on programs that support education, scientific research, infrastructure, public health and environmental protection. â€œThe (Trump) administration has already been willing to say: â€˜Weâ€™re going to increase non-defense discretionary spending ... by about 7 percent,â€™â€ Meadows, chairman of the small but influential House Freedom Caucus, said on the program. â€œNow, Democrats are saying thatâ€™s not enough, we need to give the government a pay raise of 10 to 11 percent. For a fiscal conservative, I donâ€™t see where the rationale is. ... Eventually you run out of other peopleâ€™s money,â€ he said. Meadows was among Republicans who voted in late December for their partyâ€™s debt-financed tax overhaul, which is expected to balloon the federal budget deficit and add about $1.5 trillion over 10 years to the $20 trillion national debt. â€œItâ€™s interesting to hear Mark talk about fiscal responsibility,â€ Democratic U.S. Representative Joseph Crowley said on CBS. Crowley said the Republican tax bill would require the  United States to borrow $1.5 trillion, to be paid off by future generations, to finance tax cuts for corporations and the rich. â€œThis is one of the least ... fiscally responsible bills weâ€™ve ever seen passed in the history of the House of Representatives. I think weâ€™re going to be paying for this for many, many years to come,â€ Crowley said. Republicans insist the tax package, the biggest U.S. tax overhaul in more than 30 years,  will boost the economy and job growth. House Speaker Paul Ryan, who also supported the tax bill, recently went further than Meadows, making clear in a radio interview that welfare or â€œentitlement reform,â€ as the party often calls it, would be a top Republican priority in 2018. In Republican parlance, â€œentitlementâ€ programs mean food stamps, housing assistance, Medicare and Medicaid health insurance for the elderly, poor and disabled, as well as other programs created by Washington to assist the needy. Democrats seized on Ryanâ€™s early December remarks, saying they showed Republicans would try to pay for their tax overhaul by seeking spending cuts for social programs. But the goals of House Republicans may have to take a back seat to the Senate, where the votes of some Democrats will be needed to approve a budget and prevent a government shutdown. Democrats will use their leverage in the Senate, which Republicans narrowly control, to defend both discretionary non-defense programs and social spending, while tackling the issue of the â€œDreamers,â€ people brought illegally to the country as children. Trump in September put a March 2018 expiration date on the Deferred Action for Childhood Arrivals, or DACA, program, which protects the young immigrants from deportation and provides them with work permits. The president has said in recent Twitter messages he wants funding for his proposed Mexican border wall and other immigration law changes in exchange for agreeing to help the Dreamers. Representative Debbie Dingell told CBS she did not favor linking that issue to other policy objectives, such as wall funding. â€œWe need to do DACA clean,â€ she said.  On Wednesday, Trump aides will meet with congressional leaders to discuss those issues. That will be followed by a weekend of strategy sessions for Trump and Republican leaders on Jan. 6 and 7, the White House said. Trump was also scheduled to meet on Sunday with Florida Republican Governor Rick Scott, who wants more emergency aid. The House has passed an $81 billion aid package after hurricanes in Florida, Texas and Puerto Rico, and wildfires in California. The package far exceeded the $44 billion requested by the Trump administration. The Senate has not yet voted on the aid.", 
                     "Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t even allow him to rise above the gutter long enough to wish the American citizens a happy new year!  Bishop Talbert Swan (@TalbertSwan) December 31, 2017no one likes you  Calvin (@calvinstowell) December 31, 2017Your impeachment would make 2018 a great year for America, but I ll also accept regaining control of Congress.  Miranda Yaver (@mirandayaver) December 31, 2017Do you hear yourself talk? When you have to include that many people that hate you you have to wonder? Why do the they all hate me?  Alan Sandoval (@AlanSandoval13) December 31, 2017Who uses the word Haters in a New Years wish??  Marlene (@marlene399) December 31, 2017You can t just say happy new year?  Koren pollitt (@Korencarpenter) December 31, 2017Here s Trump s New Year s Eve tweet from 2016.Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don t know what to do. Love!  Donald J. Trump (@realDonaldTrump) December 31, 2016This is nothing new for Trump. He s been doing this for years.Trump has directed messages to his  enemies  and  haters  for New Year s, Easter, Thanksgiving, and the anniversary of 9/11. pic.twitter.com/4FPAe2KypA  Daniel Dale (@ddale8) December 31, 2017Trump s holiday tweets are clearly not presidential.How long did he work at Hallmark before becoming President?  Steven Goodine (@SGoodine) December 31, 2017He s always been like this . . . the only difference is that in the last few years, his filter has been breaking down.  Roy Schulze (@thbthttt) December 31, 2017Who, apart from a teenager uses the term haters?  Wendy (@WendyWhistles) December 31, 2017he s a fucking 5 year old  Who Knows (@rainyday80) December 31, 2017So, to all the people who voted for this a hole thinking he would change once he got into power, you were wrong! 70-year-old men don t change and now he s a year older.Photo by Andrew Burton/Getty Images.",
                     "UNITED KINGDOM (Reuters) - The first operational deployment of Britain's flagship aircraft carrier HMS Queen Elizabeth to Asia will show countries such as China that Britain believes in the international law of the sea, Prime Minister Boris Johnson said on Friday. Johnson said the Carrier Strike Group, which will interact with more than 40 nations on the deployment through the Mediterranean, Indian Ocean and South China Sea to Japan, would project Britain's values as well as its military capabilities. One of the things we'll be doing clearly is showing to our friends in China that we believe in the international law of the sea, and in a confident but not a confrontational way, we will be vindicating that point, he told broadcasters on board the carrier in Portsmouth Naval Base, south England. We don't want to antagonise anybody, but we do think that the United Kingdom plays a very important role, with friends and partners, the Americans, the Dutch, the Australians, the Indians many, many others, in upholding the rule of law, the international rules-based system on which we all depend.",
                     "Good news, Wuhan’s corona virus can be cured by one bowl of freshly boiled garlic water. Old Chinese doctor has proven it’s efficacy.  Many patients has also proven this to be effective. Eight (8) cloves of chopped garlics add seven (7)cups of water and bring to boil.,  Eat and drink the boiled garlic water, overnight improvement and healing. Glad to share this."]


#Converting the sentences to sequences using a tokenizer
test_sequences2 = tokenizer.texts_to_sequences(test_sentence2)

#Padding the new sequences to make them have the same dimensions
test_padded2 = pad_sequences(test_sequences2, maxlen = max_word_length, 
                             padding = padding_type, truncating = trunc_type)

#Testing out the new padded data on a trained Bidirectional LSTM-SVM model
test_padded2 = np.array(test_padded2)
print(BILSTM_SVM_model.predict(test_padded2))

# GRU-SVM Testing Class

**Testing the GRU-SVM model by using a basic test function**

In [ ]:
test_sentence3 = [ "WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a â€œfiscal conservativeâ€ on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBSâ€™ â€œFace the Nation,â€ drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense â€œdiscretionaryâ€ spending on programs that support education, scientific research, infrastructure, public health and environmental protection. â€œThe (Trump) administration has already been willing to say: â€˜Weâ€™re going to increase non-defense discretionary spending ... by about 7 percent,â€™â€ Meadows, chairman of the small but influential House Freedom Caucus, said on the program. â€œNow, Democrats are saying thatâ€™s not enough, we need to give the government a pay raise of 10 to 11 percent. For a fiscal conservative, I donâ€™t see where the rationale is. ... Eventually you run out of other peopleâ€™s money,â€ he said. Meadows was among Republicans who voted in late December for their partyâ€™s debt-financed tax overhaul, which is expected to balloon the federal budget deficit and add about $1.5 trillion over 10 years to the $20 trillion national debt. â€œItâ€™s interesting to hear Mark talk about fiscal responsibility,â€ Democratic U.S. Representative Joseph Crowley said on CBS. Crowley said the Republican tax bill would require the  United States to borrow $1.5 trillion, to be paid off by future generations, to finance tax cuts for corporations and the rich. â€œThis is one of the least ... fiscally responsible bills weâ€™ve ever seen passed in the history of the House of Representatives. I think weâ€™re going to be paying for this for many, many years to come,â€ Crowley said. Republicans insist the tax package, the biggest U.S. tax overhaul in more than 30 years,  will boost the economy and job growth. House Speaker Paul Ryan, who also supported the tax bill, recently went further than Meadows, making clear in a radio interview that welfare or â€œentitlement reform,â€ as the party often calls it, would be a top Republican priority in 2018. In Republican parlance, â€œentitlementâ€ programs mean food stamps, housing assistance, Medicare and Medicaid health insurance for the elderly, poor and disabled, as well as other programs created by Washington to assist the needy. Democrats seized on Ryanâ€™s early December remarks, saying they showed Republicans would try to pay for their tax overhaul by seeking spending cuts for social programs. But the goals of House Republicans may have to take a back seat to the Senate, where the votes of some Democrats will be needed to approve a budget and prevent a government shutdown. Democrats will use their leverage in the Senate, which Republicans narrowly control, to defend both discretionary non-defense programs and social spending, while tackling the issue of the â€œDreamers,â€ people brought illegally to the country as children. Trump in September put a March 2018 expiration date on the Deferred Action for Childhood Arrivals, or DACA, program, which protects the young immigrants from deportation and provides them with work permits. The president has said in recent Twitter messages he wants funding for his proposed Mexican border wall and other immigration law changes in exchange for agreeing to help the Dreamers. Representative Debbie Dingell told CBS she did not favor linking that issue to other policy objectives, such as wall funding. â€œWe need to do DACA clean,â€ she said.  On Wednesday, Trump aides will meet with congressional leaders to discuss those issues. That will be followed by a weekend of strategy sessions for Trump and Republican leaders on Jan. 6 and 7, the White House said. Trump was also scheduled to meet on Sunday with Florida Republican Governor Rick Scott, who wants more emergency aid. The House has passed an $81 billion aid package after hurricanes in Florida, Texas and Puerto Rico, and wildfires in California. The package far exceeded the $44 billion requested by the Trump administration. The Senate has not yet voted on the aid.", 
                     "Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t even allow him to rise above the gutter long enough to wish the American citizens a happy new year!  Bishop Talbert Swan (@TalbertSwan) December 31, 2017no one likes you  Calvin (@calvinstowell) December 31, 2017Your impeachment would make 2018 a great year for America, but I ll also accept regaining control of Congress.  Miranda Yaver (@mirandayaver) December 31, 2017Do you hear yourself talk? When you have to include that many people that hate you you have to wonder? Why do the they all hate me?  Alan Sandoval (@AlanSandoval13) December 31, 2017Who uses the word Haters in a New Years wish??  Marlene (@marlene399) December 31, 2017You can t just say happy new year?  Koren pollitt (@Korencarpenter) December 31, 2017Here s Trump s New Year s Eve tweet from 2016.Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don t know what to do. Love!  Donald J. Trump (@realDonaldTrump) December 31, 2016This is nothing new for Trump. He s been doing this for years.Trump has directed messages to his  enemies  and  haters  for New Year s, Easter, Thanksgiving, and the anniversary of 9/11. pic.twitter.com/4FPAe2KypA  Daniel Dale (@ddale8) December 31, 2017Trump s holiday tweets are clearly not presidential.How long did he work at Hallmark before becoming President?  Steven Goodine (@SGoodine) December 31, 2017He s always been like this . . . the only difference is that in the last few years, his filter has been breaking down.  Roy Schulze (@thbthttt) December 31, 2017Who, apart from a teenager uses the term haters?  Wendy (@WendyWhistles) December 31, 2017he s a fucking 5 year old  Who Knows (@rainyday80) December 31, 2017So, to all the people who voted for this a hole thinking he would change once he got into power, you were wrong! 70-year-old men don t change and now he s a year older.Photo by Andrew Burton/Getty Images.",
                     "UNITED KINGDOM (Reuters) - The first operational deployment of Britain's flagship aircraft carrier HMS Queen Elizabeth to Asia will show countries such as China that Britain believes in the international law of the sea, Prime Minister Boris Johnson said on Friday. Johnson said the Carrier Strike Group, which will interact with more than 40 nations on the deployment through the Mediterranean, Indian Ocean and South China Sea to Japan, would project Britain's values as well as its military capabilities. One of the things we'll be doing clearly is showing to our friends in China that we believe in the international law of the sea, and in a confident but not a confrontational way, we will be vindicating that point, he told broadcasters on board the carrier in Portsmouth Naval Base, south England. We don't want to antagonise anybody, but we do think that the United Kingdom plays a very important role, with friends and partners, the Americans, the Dutch, the Australians, the Indians many, many others, in upholding the rule of law, the international rules-based system on which we all depend.",
                     "Good news, Wuhan’s corona virus can be cured by one bowl of freshly boiled garlic water. Old Chinese doctor has proven it’s efficacy.  Many patients has also proven this to be effective. Eight (8) cloves of chopped garlics add seven (7)cups of water and bring to boil.,  Eat and drink the boiled garlic water, overnight improvement and healing. Glad to share this."]


#Converting the sentences to sequences using a tokenizer
test_sequences3 = tokenizer.texts_to_sequences(test_sentence3)

#Padding the new sequences to make them have the same dimensions
test_padded3 = pad_sequences(test_sequences3, maxlen = max_word_length, 
                             padding = padding_type, truncating = trunc_type)

#Testing out the new padded data on a trained GRU-SVM model
test_padded3 = np.array(test_padded3)
print(GRU_SVM_model.predict(test_padded3))

# Bidirectional GRU-SVM Testing Class

**Testing the Bidirectional GRU-SVM model by using a basic test function**

In [ ]:
test_sentence4 = [ "WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a â€œfiscal conservativeâ€ on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBSâ€™ â€œFace the Nation,â€ drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense â€œdiscretionaryâ€ spending on programs that support education, scientific research, infrastructure, public health and environmental protection. â€œThe (Trump) administration has already been willing to say: â€˜Weâ€™re going to increase non-defense discretionary spending ... by about 7 percent,â€™â€ Meadows, chairman of the small but influential House Freedom Caucus, said on the program. â€œNow, Democrats are saying thatâ€™s not enough, we need to give the government a pay raise of 10 to 11 percent. For a fiscal conservative, I donâ€™t see where the rationale is. ... Eventually you run out of other peopleâ€™s money,â€ he said. Meadows was among Republicans who voted in late December for their partyâ€™s debt-financed tax overhaul, which is expected to balloon the federal budget deficit and add about $1.5 trillion over 10 years to the $20 trillion national debt. â€œItâ€™s interesting to hear Mark talk about fiscal responsibility,â€ Democratic U.S. Representative Joseph Crowley said on CBS. Crowley said the Republican tax bill would require the  United States to borrow $1.5 trillion, to be paid off by future generations, to finance tax cuts for corporations and the rich. â€œThis is one of the least ... fiscally responsible bills weâ€™ve ever seen passed in the history of the House of Representatives. I think weâ€™re going to be paying for this for many, many years to come,â€ Crowley said. Republicans insist the tax package, the biggest U.S. tax overhaul in more than 30 years,  will boost the economy and job growth. House Speaker Paul Ryan, who also supported the tax bill, recently went further than Meadows, making clear in a radio interview that welfare or â€œentitlement reform,â€ as the party often calls it, would be a top Republican priority in 2018. In Republican parlance, â€œentitlementâ€ programs mean food stamps, housing assistance, Medicare and Medicaid health insurance for the elderly, poor and disabled, as well as other programs created by Washington to assist the needy. Democrats seized on Ryanâ€™s early December remarks, saying they showed Republicans would try to pay for their tax overhaul by seeking spending cuts for social programs. But the goals of House Republicans may have to take a back seat to the Senate, where the votes of some Democrats will be needed to approve a budget and prevent a government shutdown. Democrats will use their leverage in the Senate, which Republicans narrowly control, to defend both discretionary non-defense programs and social spending, while tackling the issue of the â€œDreamers,â€ people brought illegally to the country as children. Trump in September put a March 2018 expiration date on the Deferred Action for Childhood Arrivals, or DACA, program, which protects the young immigrants from deportation and provides them with work permits. The president has said in recent Twitter messages he wants funding for his proposed Mexican border wall and other immigration law changes in exchange for agreeing to help the Dreamers. Representative Debbie Dingell told CBS she did not favor linking that issue to other policy objectives, such as wall funding. â€œWe need to do DACA clean,â€ she said.  On Wednesday, Trump aides will meet with congressional leaders to discuss those issues. That will be followed by a weekend of strategy sessions for Trump and Republican leaders on Jan. 6 and 7, the White House said. Trump was also scheduled to meet on Sunday with Florida Republican Governor Rick Scott, who wants more emergency aid. The House has passed an $81 billion aid package after hurricanes in Florida, Texas and Puerto Rico, and wildfires in California. The package far exceeded the $44 billion requested by the Trump administration. The Senate has not yet voted on the aid.", 
                     "Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t even allow him to rise above the gutter long enough to wish the American citizens a happy new year!  Bishop Talbert Swan (@TalbertSwan) December 31, 2017no one likes you  Calvin (@calvinstowell) December 31, 2017Your impeachment would make 2018 a great year for America, but I ll also accept regaining control of Congress.  Miranda Yaver (@mirandayaver) December 31, 2017Do you hear yourself talk? When you have to include that many people that hate you you have to wonder? Why do the they all hate me?  Alan Sandoval (@AlanSandoval13) December 31, 2017Who uses the word Haters in a New Years wish??  Marlene (@marlene399) December 31, 2017You can t just say happy new year?  Koren pollitt (@Korencarpenter) December 31, 2017Here s Trump s New Year s Eve tweet from 2016.Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don t know what to do. Love!  Donald J. Trump (@realDonaldTrump) December 31, 2016This is nothing new for Trump. He s been doing this for years.Trump has directed messages to his  enemies  and  haters  for New Year s, Easter, Thanksgiving, and the anniversary of 9/11. pic.twitter.com/4FPAe2KypA  Daniel Dale (@ddale8) December 31, 2017Trump s holiday tweets are clearly not presidential.How long did he work at Hallmark before becoming President?  Steven Goodine (@SGoodine) December 31, 2017He s always been like this . . . the only difference is that in the last few years, his filter has been breaking down.  Roy Schulze (@thbthttt) December 31, 2017Who, apart from a teenager uses the term haters?  Wendy (@WendyWhistles) December 31, 2017he s a fucking 5 year old  Who Knows (@rainyday80) December 31, 2017So, to all the people who voted for this a hole thinking he would change once he got into power, you were wrong! 70-year-old men don t change and now he s a year older.Photo by Andrew Burton/Getty Images.",
                     "UNITED KINGDOM (Reuters) - The first operational deployment of Britain's flagship aircraft carrier HMS Queen Elizabeth to Asia will show countries such as China that Britain believes in the international law of the sea, Prime Minister Boris Johnson said on Friday. Johnson said the Carrier Strike Group, which will interact with more than 40 nations on the deployment through the Mediterranean, Indian Ocean and South China Sea to Japan, would project Britain's values as well as its military capabilities. One of the things we'll be doing clearly is showing to our friends in China that we believe in the international law of the sea, and in a confident but not a confrontational way, we will be vindicating that point, he told broadcasters on board the carrier in Portsmouth Naval Base, south England. We don't want to antagonise anybody, but we do think that the United Kingdom plays a very important role, with friends and partners, the Americans, the Dutch, the Australians, the Indians many, many others, in upholding the rule of law, the international rules-based system on which we all depend.",
                     "Good news, Wuhan’s corona virus can be cured by one bowl of freshly boiled garlic water. Old Chinese doctor has proven it’s efficacy.  Many patients has also proven this to be effective. Eight (8) cloves of chopped garlics add seven (7)cups of water and bring to boil.,  Eat and drink the boiled garlic water, overnight improvement and healing. Glad to share this."]


# Converting the sentences to sequences using tokenizer
test_sequences4 = tokenizer.texts_to_sequences(test_sentence4)

# padding the new sequences to make them have same dimensions
test_padded4 = pad_sequences(test_sequences4, maxlen = max_word_length, 
                             padding = padding_type, truncating = trunc_type)

#Testing out the new padded data on a trained BIGRU-SVM model
test_padded4 = np.array(test_padded4)
print(BIGRU_SVM_model.predict(test_padded4))

# Conclusion

The LSTM-SVM model is the best model to use when trying to detect true or false news articles. This model was able to understand fake news articles and true news articles compared to other models which failed to distinguish between two true and two fake news articles. This model has the benefit of using its three gates to its maximum potential and being a powerful feed-forward network. These gates are the input gate which inputs the information, forget gate which discards any biased or low result values, and the output gate which outputs the result combined with the SVM optimal hyperplanes which are designed for the separation of training data without errors and soft margins which allows for an analytic treatment of learning with errors on the training set compared to the Bidirectional models which are complex since it came to go forward and backward with more training time. In the end, the complexity of this model allows it to differentiate between what news articles are real or fabricated.